Excelente ideia! Separar as alterações por trechos de código tornará a compreensão muito mais clara.

Vou apresentar cada trecho do seu código original que será substituído ou modificado, seguido pelo novo código e uma explicação detalhada.

1. Invocação do LLM e Pós-processamento (O Coração da Mudança)

Esta é a parte mais crítica do pipeline, onde a inferência do LLM é realizada e as respostas são processadas.

Código Original:

# ... (dentro do bloco `if df_spk.count() > 0:`) ...

llm_client = openai.OpenAI(api_key=DATABRICKS_TOKEN,
                       base_url="https://dbc-d80f50a9-af23.cloud.databricks.com/serving-endpoints"
                       )
descricao_agente = "Atue como um médico oncologista especialista em laudos de mamografia."

# Coleta os dados localmente
df_local = df_spk.select("ficha","id_item","id_subitem","id_cliente","dth_pedido","dth_resultado", "sigla_exame", "laudo_tratado","linha_cuidado","_datestamp").toPandas()

# Aplica o LLM localmente
df_local["resposta_llm"] = batch_generate(descricao_agente, df_local["laudo_tratado"].tolist(), llm_client, batch_size=25)
df_local = df_local.join(df_local["resposta_llm"].apply(limpar_e_converter).apply(pd.Series))

# ... (restante do código que usa df_local) ...

# Funções auxiliares para o LLM
def prompt_laudo(laudo_texto: str) -> str:
# ... (código da função prompt_laudo) ...

def generate(descricao_agente:str, laudo:str, llm_client) -> str:
# ... (código da função generate) ...

def batch_generate(descricao_agente, laudos, llm_client, batch_size=25):
# ... (código da função batch_generate) ...

def limpar_e_converter(item):
# ... (código da função limpar_e_converter) ...


Problema/Contexto:

df_spk.toPandas(): Coleta todos os dados do Spark para o nó driver, o que é um gargalo de performance e escalabilidade para grandes volumes.
batch_generate e generate: Apesar do nome, batch_generate chama o LLM sequencialmente para cada laudo dentro de um lote, e ainda inclui um time.sleep(0.5). Isso é extremamente lento e ineficiente.
limpar_e_converter: É uma função Python que processa as respostas JSON localmente, após a coleta dos dados.
Gerenciamento do Cliente LLM: O cliente openai.OpenAI é inicializado e gerenciado manualmente.

Novo Código:

import json # Necessário para json.dumps
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# --- Nova definição do prompt como template de string ---
# Esta função agora retorna um template de string que será preenchido pelo Spark
def prompt_laudo_template() -> str:
prompt = """A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

Laudo clínico:
\"\"\"{laudo_texto}\"\"\"

### Critérios de extração:

- **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

- **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

- **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
- "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
- "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
- "HER-2 - ESCORE 3+" → "POSITIVO"

- **Ki-67 (%)**: retorne o valor numérico da porcentagem de positividade do KI-67, caso seja um intervalo de valores retorne o intervalo com valor mínimo e valor máximo separado por um hífem.

- **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

### Saída esperada (dicionário Python válido):
```python
{{
"receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
"receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
"status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
"ki67_percentual": float |  0,
"status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
}}


""" return prompt.strip()

--- Schema para o JSON de saída do LLM (usado por from_json) ---

llm_output_schema = StructType([ StructField("receptor_estrogeno", StringType(), True), StructField("receptor_progesterona", StringType(), True), StructField("status_her2", StringType(), True), StructField("ki67_percentual", StringType(), True), # Manter como StringType para lidar com "NÃO INFORMADO" ou intervalos StructField("status_ck5_6", StringType(), True), ])

--- Bloco principal de execução (dentro do if df_spk.count() > 0:) ---
Nome do modelo Foundation Model no Databricks

llm_model_name = "databricks-llama-4-maverick" # Ou o modelo que você estiver usando

O prompt_laudo_template() é uma string Python.
Precisamos escapar as aspas para SQL e usar CONCAT para injetar o laudo_tratado.
O json.dumps é usado para escapar o template do prompt para ser seguro em SQL.
O .replace é para o placeholder do laudo.

escaped_prompt_template = json.dumps(prompt_laudo_template()).replace('"{{laudo_texto}}"', "laudo_tratado")

Usa ai_query para invocar o LLM de forma distribuída

df_with_llm_raw_responses = df_spk.withColumn( "llm_raw_response", F.expr(f""" ai_query( '{llm_model_name}', CONCAT({escaped_prompt_template}), temperature => 0.0, max_tokens => 4000, top_p => 0.75, frequency_penalty => 0.0, presence_penalty => 0.0 ) """) )

Converte as strings JSON das respostas do LLM em colunas estruturadas

df_llm_parsed = df_with_llm_raw_responses.withColumn( "llm_parsed_output", F.from_json(F.col("llm_raw_response"), llm_output_schema) ).select( "*", # Mantém todas as colunas originais F.col("llm_parsed_output.receptor_estrogeno").alias("receptor_estrogeno"), F.col("llm_parsed_output.receptor_progesterona").alias("receptor_progesterona"), F.col("llm_parsed_output.status_her2").alias("status_her2"), F.col("llm_parsed_output.ki67_percentual").alias("ki67_percentual"), F.col("llm_parsed_output.status_ck5_6").alias("status_ck5_6") ).drop("llm_raw_response", "llm_parsed_output") # Remove as colunas intermediárias

Garante que ki67_percentual seja float para a classificação

df_llm_parsed = df_llm_parsed.withColumn( "ki67_percentual_float", F.when( F.col("ki67_percentual").cast(DoubleType()).isNotNull(), F.col("ki67_percentual").cast(DoubleType()) ).otherwise(0.0) # Se não for um número válido, assume 0.0 )

... (restante do código de classificação, que usa df_llm_parsed) ...

**Explicação da Mudança:**

*   **Remoção de `df_spk.toPandas()`**: Os dados permanecem no `DataFrame` Spark, garantindo processamento distribuído.
*   **Remoção de `generate`, `batch_generate`, `limpar_e_converter`**: Toda a lógica de invocação e parsing do LLM é substituída por uma única expressão Spark usando `ai_query` e `from_json`.
*   **`ai_query`**: Esta função Spark SQL é a chave. Ela invoca o Foundation Model configurado no Databricks, passando o prompt (construído dinamicamente com `CONCAT` e o `laudo_tratado` de cada linha) e os parâmetros do modelo. O Databricks gerencia o batching, retries e a escalabilidade automaticamente.
*   **`prompt_laudo_template()`**: A função `prompt_laudo` foi adaptada para retornar um template de string. Este template é então injetado na expressão `ai_query` de forma segura usando `json.dumps` e `replace` para o placeholder `laudo_texto`.
*   **`from_json()`**: Substitui a função `limpar_e_converter`. Ele parseia a string JSON retornada pelo LLM diretamente no Spark, usando o `llm_output_schema` para tipagem e estruturação. Isso é muito mais eficiente e robusto.
*   **`ki67_percentual_float`**: A conversão para float agora é feita diretamente no Spark, com tratamento para valores não numéricos.

---

### 2. Funções de Extração Heurística e Avaliação

Estas funções são para gerar "pseudo-ground truth" e comparar com a saída do LLM. Elas não fazem parte do pipeline de inferência principal, mas são importantes para a validação.

**Código Original:**

```python
# =================================================================
# Funções de extração heurística (pseudo-gold) para o novo prompt
# =================================================================

def extrai_receptor_estrogeno(txt: str) -> str:
# ... (código da função) ...

def extrai_receptor_progesterona(txt: str) -> str:
# ... (código da função) ...

def extrai_status_her2(txt: str) -> str:
# ... (código da função) ...

def extrai_ki67_percentual(txt: str) -> float:
# ... (código da função) ...

def extrai_status_ck5_6(txt: str) -> str:
# ... (código da função) ...

# ==========================================================================
# Função de avaliação sem ground truth completo (novo conjunto de campos)
# ==========================================================================

def avalia_extracao_sem_ground_truth_imuno(laudo_texto: str, json_modelo: dict):
# ... (código da função) ...

from collections import Counter

def agrega_resultados_dinamico(lista_comparacoes):
# ... (código da função) ...


Problema/Contexto:

Nenhum problema intrínseco a estas funções. Elas são ferramentas de validação.

Novo Código:

Nenhum. Estas funções permanecem inalteradas e devem ser usadas para seu propósito original de avaliação.

Explicação da Mudança:

Estas funções não são usadas no novo pipeline de inferência com ai_query.
Para usá-las para avaliar o desempenho do LLM, você precisaria coletar uma amostra do df_llm_parsed (ou df_final_classif) para o nó driver (usando .limit(N).toPandas()) e então aplicar essas funções localmente. Isso evita coletar todo o dataset, que é o problema que estamos resolvendo.
3. Função parse_json_string

Código Original:

def parse_json_string(s):
if isinstance(s, str):
    try:
        return ast.literal_eval(s)
    except Exception:
        return {}
return s


Problema/Contexto:

Esta função era usada para parsear strings JSON.

Novo Código:

Removida.

Explicação da Mudança:

A funcionalidade de parsing de JSON é agora tratada de forma mais eficiente e distribuída pela função Spark F.from_json(), que opera diretamente nos DataFrames. Portanto, esta função Python não é mais necessária.

Com estas alterações, seu pipeline se torna nativamente distribuído, escalável e otimizado para o uso de Foundation Models no Databricks, abordando diretamente as questões de custo e performance.

In [ ]:
# --- Bloco de avaliação de métricas (após a geração de df_llm_parsed ou df_final_classif) ---

# Para realizar a avaliação, você precisará coletar uma amostra dos dados para o driver.
# Isso é aceitável para avaliação, mas não para o pipeline de inferência principal.
# Ajuste o `limit(N)` conforme a quantidade de dados que você deseja avaliar.
df_sample_for_eval = df_llm_parsed.limit(100).toPandas() # Coleta uma amostra para avaliação local

lista_laudos_eval = df_sample_for_eval["laudo_tratado"].tolist()

# Prepara a saída do LLM para a função de avaliação
# O `json_model_output` deve ser um dicionário Python para cada linha
lista_json_modelo_eval = []
for index, row in df_sample_for_eval.iterrows():
    lista_json_modelo_eval.append({
        "receptor_estrogeno": row["receptor_estrogeno"],
        "receptor_progesterona": row["receptor_progesterona"],
        "status_her2": row["status_her2"],
        "ki67_percentual": row["ki67_percentual"], # Use a string original do LLM
        "status_ck5_6": row["status_ck5_6"]
    })

lista_pseudo_gold_eval = []
lista_comparacoes_eval = []

# As funções avalia_extracao_sem_ground_truth_imuno e agrega_resultados_dinamico
# devem estar definidas em células anteriores.
for laudo_txt, json_mod in zip(lista_laudos_eval, lista_json_modelo_eval):
    json_heu, comp = avalia_extracao_sem_ground_truth_imuno(laudo_txt, json_mod)
    lista_pseudo_gold_eval.append(json_heu)
    lista_comparacoes_eval.append(comp)

# Agrega os resultados das comparações
json_metricas = agrega_resultados_dinamico(lista_comparacoes_eval)

print("Métricas de Avaliação:")
print(json.dumps(json_metricas, indent=2))

# Opcional: Para visualizar os resultados detalhados da avaliação
# df_metrics_eval = pd.DataFrame()
# df_metrics_eval["laudos"] = lista_laudos_eval
# df_metrics_eval["json_modelo"] = lista_json_modelo_eval
# df_metrics_eval["json_heuristico"] = lista_pseudo_gold_eval
# df_metrics_eval["comparacoes"] = lista_comparacoes_eval
# display(df_metrics_eval)



casa


In [ ]:
# --- Funções auxiliares para exportação de Excel (manter estas funções em uma célula) ---

def salvar_excel(df, nome_arquivo):
    """
    Salva um DataFrame em um arquivo Excel.

    Args:
        df (pandas.DataFrame): DataFrame a ser salvo.
        nome_arquivo (str): Nome do arquivo Excel.
    """

    # After installing, save the DataFrame to Excel again
    df.to_excel(nome_arquivo, index=False)

def conta_marcadores(txt: str, marcador: str) -> int:
    """
    Conta quantas vezes uma marcador aparece no texto, ignorando maiúsculas/minúsculas.
    """
    marcadores = {
        'estrogenio': r"\b(receptor\s+de\s+estr[oó]g(?:[eê]nio|eno))\b",
        'progesterona': r"\b(receptor\s+de\s+progesterona)\b",
        'her2': r"\bher2\b",
        'ck5_6': r"\bck5\s*\/\s*6\b",
        'ki67': r"\bki[-\s]?67\b"
    }
    pattern = marcadores.get(marcador)
    if not pattern:
        raise ValueError(f"marcador '{marcador}' não reconhecida. Use uma das seguintes: {list(marcadores.keys())}")

    matches = re.findall(pattern, txt, flags=re.IGNORECASE)
    return len(matches)

def pega_texto_marcador(txt: str, sigla: str) -> str: # Alterado para retornar str, não int
    """
    Pega o texto que vem depois do marcador, limitado em 100 caracteres.
    """
    siglas = {
        'estrogenio': r"\b(receptor\s+de\s+estr[oó]g(?:[eê]nio|eno))\b",
        'progesterona': r"\b(receptor\s+de\s+progesterona)\b",
        'her2': r"\bher2\b",
        'ck5_6': r"\bck5\s*\/\s*6\b",
        'ki67': r"\bki[-\s]?67\b"
    }
    pattern = siglas.get(sigla)
    if not pattern:
        raise ValueError(f"sigla '{sigla}' não reconhecida. Use uma das seguintes: {list(siglas.keys())}")

    match = re.search(pattern, txt, flags=re.IGNORECASE)
    if match:
        # Pega o texto após o marcador, limitado a 100 caracteres
        start = match.end()
        end = start + 100
        return txt[start:end].strip()
    return ""

# --- Bloco para gerar a planilha de comparação (em uma nova célula) ---

# df_sample_for_eval e lista_comparacoes_eval devem ter sido gerados na célula de avaliação anterior.
# Se a célula de avaliação não for executada, este bloco falhará.

if 'df_sample_for_eval' in locals() and not df_sample_for_eval.empty:
    # Cria uma cópia do DataFrame de amostra para não modificar o original
    df_laura = df_sample_for_eval.copy()

# Normaliza a lista de comparações para adicionar as colunas de acerto
# lista_comparacoes_eval deve vir da célula de avaliação
if 'lista_comparacoes_eval' in locals() and lista_comparacoes_eval:
    resultados_expandidos = pd.json_normalize(lista_comparacoes_eval)
    # Adiciona um prefixo para evitar conflito de nomes de colunas
    resultados_expandidos = resultados_expandidos.add_prefix('comparacao_')
    df_laura = pd.concat(
        [df_laura.reset_index(drop=True), resultados_expandidos.reset_index(drop=True)],
        axis=1
    )

    # Renomeia as colunas de acerto para o formato esperado
    df_laura = df_laura.rename(columns={
        'comparacao_receptor_estrogeno.acertou': 'receptor_estrogeno.acertou',
        'comparacao_receptor_progesterona.acertou': 'receptor_progesterona.acertou',
        'comparacao_status_her2.acertou': 'status_her2.acertou',
        'comparacao_ki67_percentual.acertou': 'ki67_percentual.acertou',
        'comparacao_status_ck5_6.acertou': 'status_ck5_6.acertou'
    })

    # Contagem de quantas vezes cada sigla aparece no laudo
    siglas = ['estrogenio', 'progesterona', 'her2', 'ck5_6', 'ki67']
    for sigla in siglas:
        df_laura['ctg_' + sigla] = df_laura['laudo_tratado'].apply(lambda x: conta_marcadores(x, sigla))
        # Adiciona o texto do marcador para análise
        df_laura['txt_' + sigla] = df_laura['laudo_tratado'].apply(lambda x: pega_texto_marcador(x, sigla))

    # Insere coluna de erro genérico
    df_laura['erro'] = df_laura.apply(lambda x:
                                      (x.get('ki67_percentual.acertou', False) == False) or
                                      (x.get('receptor_estrogeno.acertou', False) == False) or
                                      (x.get('receptor_progesterona.acertou', False) == False) or
                                      (x.get('status_her2.acertou', False) == False) or
                                      (x.get('status_ck5_6.acertou', False) == False), axis=1)

    # Salva o DataFrame em Excel
    nome_arquivo_excel = "/dbfs/FileStore/shared_uploads/seu_email/analise_laudos_comparacao.xlsx" # Ajuste o caminho
    salvar_excel(df_laura, nome_arquivo_excel)
    print(f"Planilha de comparação salva em: {nome_arquivo_excel}")
    display(df_laura) # Para visualizar no notebook
else:
    print("Lista de comparações vazia ou não encontrada. Não foi possível gerar a planilha de comparação.")
else:
print("DataFrame de amostra para avaliação (df_sample_for_eval) vazio ou não encontrado. Não foi possível gerar a planilha de comparação.")

## INCONSISTÊNCIAS IDENTIFICADAS NOS LAUDOS DE IMUNO-HISTOQUÍMICA
### RE - RECEPTOR DE ESTRÓGENO
- MAIS DE UM RESULTADOS NO MESMO LAUDO (13% erro confirmado - 84% para investigação granular)


### RP - RECEPTOR DE PROGESTERONA
- NEGATIVO E POSITIVO NA MESMA FRASE (3% de erro confirmado)
    - NEGATIVO NO CARCINOMA MAMÁRIO INVASIVO CRIBRIFORME E POSITIVO COM FORTE INTENSIDADE EM 98% DAS CÉLULAS DO CARCINOMA LOBULAR INVASIVO
    - NEGATIVO NAS CÉLULAS DA NEOPLASIA INVASIVA E POSITIVO COM INTENSIDADE MODERADA A FORTE EM 30% DAS CÉLULAS DO COMPONENTE DUCTAL 'IN SITU'
    - NEGATIVO (CONTROLE INTERNO POSITIVO)
    - NEGATIVO NAS CÉLULAS NEOPLÁSICAS DO FOCO DE INVASÃO MAIOR / POSITIVO COM INTENSIDADE MODERADA EM 40% DAS CÉLULAS DO CDIS
    - NEGATIVO NA CÉLULAS NEOPLÁSICAS (CONTROLE INTERNO POSITIVO)
    - NEGATIVO NAS CÉLULAS NEOPLÁSICAS (bloco A1; controle externo positivo)
- MAIS DE UM RESULTADOS NO MESMO LAUDO

### HER2
- MAIS DE UM RESULTADOS NO MESMO LAUDO (55% erro confirmado)

### CK5/6
- MAIS DE UM RESULTADOS NO MESMO LAUDO
- TEXTO DESPADRONIZADO (SEM EXPECIFICAR POSITIVO OU NEGATIVO) - (0,8% de erro confirmado)
    - PERDA DE EXPRESSÃO NAS CELULAS DOS FOCOS DE AEP
    - POSITIVO EM 20% DAS CÉLULAS NEOPLÁSICAS (DIFERENCIAÇÃO ESCAMOSA)
    - PERDA DE EXPRESSÃO NOS FOCOS DE ATIPIA E NO CLIS
    - PERDA DE EXPRESSÃO NO CDIS
    - PERDA DE EXPRESSÃO NAS CÉLULAS NEOPLÁSICAS DO CDIS (PA) E NOS FOCOS DE HLA E ATIPIA PLANA
    - PERDA DE EXPRESSÃO NOS FOCOS DE ATIPIA DA LESÃO PAPILÍFERA
    - EXPRESSÃO REDUZIDA EM ALGUMAS CÉLULAS EPITELIAIS
    - PERDA DE EXPRESSÃO NOS FOCOS DE CDIS E HLA DAS AMOSTRAS B2 e B3 (EXPRESSÃO NORMAL NA AMOSTRA A1)
    - PERDA DE EXPRESSÃO NAS CÉLULAS ATÍPICAS (A1, A2 e A3)
    - PERDA DE EXPRESSÃO NOS FOCOS DE CDIS E CLIS NAS AMOSTRAS

### KI67
- MAIS DE UM RESULTADOS NO MESMO LAUDO (13% erro confirmado - 84% para investigação granular)
- TEXTO COM FAIXA DE PERCENTUAL - (2% de erro confirmado)
    - proliferação celular estimada em 15-20%.
    - POSITIVO EM 20 A 30% DAS CÉLULAS NEOPLÁSICAS
    - POSITIVO EM 3-5% DAS CÉLULAS EPITELIAIS DE INTERESSE
- MAIS DE UM RESULTADO NA MESMA LINHA DE TEXTO
    - NÃO AVALIÁVEL NO FOCO DE MICROINVASÃO / POSITIVO EM 50% DAS CÉLULAS NEOPLÁSICAS DO CDIS Anti-KI-6
    - POSITIVO EM 40% DAS CÉLULAS DO CARCINOMA MAMÁRIO INVASIVO CRIBRIFORME E POSITIVO EM 06% DAS CÉLULAS DO CARCINOMA LOBULAR INVASIVO.

```python
def prompt_laudo_template() -> str:
# REMOVER O 'f' ANTES DAS ASPAS TRIPLAS
prompt = """A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

Laudo clínico:
\"\"\"{laudo_texto}\"\"\"

### Critérios de extração:

- **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

- **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

- **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
- "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
- "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
- "HER-2 - ESCORE 3+" → "POSITIVO"

- **Ki-67 (%)**: retorne o valor numérico da porcentagem de positividade do KI-67, caso seja um intervalo de valores retorne o intervalo com valor mínimo e valor máximo separado por um hífem.

- **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

### Saída esperada (dicionário Python válido):
```python
{ # Remover as chaves duplas aqui também
"receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
"receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
"status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
"ki67_percentual": float |  0,
"status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
}
""" 
return prompt.strip()

In [ ]:
def prompt_laudo_template() -> str:
    # REMOVER O 'f' ANTES DAS ASPAS TRIPLAS
    prompt = """A seguir está um laudo médico de mamografia, conforme indicado abaixo. Se alguma informação não estiver presente no texto, retorne "NÃO INFORMADO". Sempre retorne apenas o dicionário Python.

    Laudo clínico:
    \"\"\"{laudo_texto}\"\"\"

    ### Critérios de extração:

    - **Receptor de Estrogênio**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Receptor de Progesterona**: retorne se é "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO".

    - **Status do HER-2**: retorne se o Status do HER-2 é "NEGATIVO", "INCONCLUSIVO", "POSITIVO" ou "NÃO INFORMADO". Com base no score seguindo as regras:
    - "HER-2 - ESCORE 0" ou "1+" → "NEGATIVO"
    - "HER-2 - ESCORE 2+" → "INCONCLUSIVO"
    - "HER-2 - ESCORE 3+" → "POSITIVO"

    - **Ki-67 (%)**: retorne o valor numérico da porcentagem de positividade do KI-67, caso seja um intervalo de valores retorne o intervalo com valor mínimo e valor máximo separado por um hífem.

    - **Status do CK5/6**: retorne "POSITIVO", "NEGATIVO" ou "NÃO INFORMADO" do Status do CK5/6.

    ### Saída esperada (dicionário Python válido):
    ```python
    { # Remover as chaves duplas aqui também
    "receptor_estrogeno": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "receptor_progesterona": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO",
    "status_her2": "POSITIVO" | "POSITIVO" | "INCONCLUSIVO" |  "NÃO INFORMADO",
    "ki67_percentual": float |  0,
    "status_ck5_6": "POSITIVO" | "NEGATIVO" |  "NÃO INFORMADO"
    }
    """ 
    return prompt.strip()
